<a href="https://colab.research.google.com/github/sanjaypriyadarsan/Emotion-Detection-from-Uploaded-Images/blob/main/Emtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os

# Unzip the uploaded archive
with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")  # Extract to "dataset" folder

# Check contents
os.listdir("dataset")


['test', 'train']

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])


In [ ]:
# Paths to folders
train_path = "dataset/train"
test_path = "dataset/test"

# Load with ImageFolder
train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Check class labels
print("Classes:", train_dataset.classes)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 7)  # 7 emotion classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> [32x24x24]
        x = self.pool(F.relu(self.conv2(x)))  # -> [64x12x12]
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [ ]:
import torch
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")


Epoch [1/30], Loss: 762.8859
Epoch [2/30], Loss: 686.8965
Epoch [3/30], Loss: 646.3395
Epoch [4/30], Loss: 623.7537
Epoch [5/30], Loss: 605.5944
Epoch [6/30], Loss: 593.6040
Epoch [7/30], Loss: 582.4293
Epoch [8/30], Loss: 575.3682
Epoch [9/30], Loss: 566.1083
Epoch [10/30], Loss: 561.1507
Epoch [11/30], Loss: 553.2549
Epoch [12/30], Loss: 545.0024
Epoch [13/30], Loss: 540.9138
Epoch [14/30], Loss: 534.9259
Epoch [15/30], Loss: 533.1064
Epoch [16/30], Loss: 528.8417
Epoch [17/30], Loss: 521.7092
Epoch [18/30], Loss: 518.0799
Epoch [19/30], Loss: 515.2642
Epoch [20/30], Loss: 510.5339
Epoch [21/30], Loss: 506.9536
Epoch [22/30], Loss: 502.2824
Epoch [23/30], Loss: 500.2564
Epoch [24/30], Loss: 496.1154
Epoch [25/30], Loss: 494.2376
Epoch [26/30], Loss: 493.0525
Epoch [27/30], Loss: 490.4793
Epoch [28/30], Loss: 489.3914
Epoch [29/30], Loss: 485.5946
Epoch [30/30], Loss: 483.9356


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [ ]:
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))


Classification Report:

              precision    recall  f1-score   support

       angry       0.42      0.44      0.43       958
     disgust       0.45      0.18      0.26       111
        fear       0.46      0.22      0.30      1024
       happy       0.66      0.84      0.74      1774
     neutral       0.48      0.53      0.51      1233
         sad       0.41      0.42      0.41      1247
    surprise       0.72      0.65      0.68       831

    accuracy                           0.54      7178
   macro avg       0.51      0.47      0.48      7178
weighted avg       0.53      0.54      0.52      7178



In [ ]:
torch.save(model.state_dict(), "emotion_cnn.pth")


In [ ]:
from google.colab import files
files.download("emotion_cnn.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 479.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok


In [ ]:
# model.py
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


Writing model.py


In [ ]:
# app.py
%%writefile app.py
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np
import mediapipe as mp
from model import EmotionCNN

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionCNN().to(device)
model.load_state_dict(torch.load('emotion_cnn.pth', map_location=device))
model.eval()

# Emotion labels
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Preprocessing for prediction
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

# Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Predict emotion
def predict_emotion(face_img):
    face_img = Image.fromarray(face_img)
    face_tensor = transform(face_img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(face_tensor)
        _, predicted = torch.max(output, 1)
    return EMOTIONS[predicted.item()]

# App UI
st.title("🧠 Emotion Detection from Image")
st.write("Upload an image, and I’ll detect the face and predict the emotion!")

uploaded_file = st.file_uploader("Upload a JPG/PNG image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    img_array = np.array(image)
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    # Face detection
    results = face_detector.process(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))

    if results.detections:
        for det in results.detections:
            bboxC = det.location_data.relative_bounding_box
            h, w, _ = img_rgb.shape
            x, y, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
            face = img_rgb[y:y + h_box, x:x + w_box]

            try:
                emotion = predict_emotion(face)
                st.image(img_rgb, caption="Detected Face", channels="BGR")
                st.success(f"Predicted Emotion: **{emotion.upper()}**")
            except:
                st.warning("Couldn't process face properly. Try another image.")
    else:
        st.warning("No face detected. Please upload a clearer image.")


Writing app.py


In [ ]:
!pip install mediapipe

In [ ]:
!pip install easyocr

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 6s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.187.151.240
⠙⠹⠸your url is: https://twelve-jokes-battle.loca.lt
